In [1]:
#데이터 및 튜토리얼 출처 : https://github.com/lovit/
rawtext_fname = 'C:/Users/hp/Desktop/navermovie_comments/data/data_small.txt'
with open(rawtext_fname, encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

45290	크리스토퍼 놀란 에게 우리는 놀란 다	10
45290	인셉션 정말 흥미진진하게 봤었고 크리스토퍼 놀란 감독님 신작 인터스텔라도 이번주 일요일에 보러갑니다 완전 기대중	10
45290	놀란이면 무조건 봐야 된다 왜냐하면 모든 작품을 다 히트 쳤으니깐	10
45290	나는 감탄할 준비가 되어있다	10
45290	얘들아 오늘나오는거지 밤에 ㅋㅋ 오늘 보러가야겟다	10


In [2]:
def get_movie_comments(n_limit=-1):

    fname = 'C:/Users/hp/Desktop/navermovie_comments/data/data_small.txt'

    with open(fname, encoding='utf-8') as f:
        idxs = []
        scores = []
        texts = []

        for i, doc in enumerate(f):
            if n_limit > 0 and i >= n_limit:
                break

            try:
                idx, text, score = doc.strip().split('\t')
                idxs.append(idx)
                texts.append(text)
                scores.append(int(score))
            except Exception as e:
                print(e)
                continue

    return idxs, texts, scores


In [3]:
idxs, texts, scores = get_movie_comments(n_limit=5000)

In [4]:
from collections import defaultdict
from pprint import pprint
from konlpy.tag import Komoran

#명사 추출. type은 list of str
komoran = Komoran()
komoran.nouns('이건 테스트입니다')

['이건', '테스트']

In [5]:
# collections.Counter 를 이용하여 한 번에 명사의 개수를 세어 봅시다.


from collections import Counter

noun_counter = Counter(
    [noun for text in texts for noun in komoran.nouns(text)]
)

print('num of nouns: %d' % len(noun_counter))

num of nouns: 3290


In [6]:
#term frequency matrix 를 만들 때, 거의 등장하지 않는 단어들은 의미가 없습니다. min count 를 설정하여 각 threshold 별로 몇 개의 명사가 살아남는지 알아봅니다.

for min_count in [2, 3, 5, 10]:
    _counter = {
        word for word, freq in noun_counter.items()
        if freq >= min_count
    }
    print('num of nouns (min_count = %d): %d' % (
        min_count, len(_counter)))

num of nouns (min_count = 2): 1695
num of nouns (min_count = 3): 1181
num of nouns (min_count = 5): 758
num of nouns (min_count = 10): 417


우리는 2번 이상 나오는 1695 개의 명사를 이용하여 term frequency matrix 를 만들어 봅니다. 우리는 전체 문서에서의 빈도수가 2 이상인 명사만을 return 하는 토크나이저를 만들 것입니다.

Komoran.nouns() 와 custom_tokenizer의 결과가 달라짐을 확인할 수 있습니다.

In [7]:
noun_dict = {
    word for word, freq in noun_counter.items()
    if freq >= 2
}

def custom_tokenizer(doc):
    return [word for word in komoran.nouns(doc) if word in noun_dict]

print(komoran.nouns(texts[1]))
print(custom_tokenizer(texts[1]))

['인셉션', '크리스토퍼', '감독', '신작', '인터스텔라', '이번', '주', '일요일', '완전', '기대', '중']
['인셉션', '크리스토퍼', '감독', '신작', '인터스텔라', '이번', '주', '완전', '기대', '중']


In [50]:
custom_tokenizer

<function __main__.custom_tokenizer(doc)>

In [8]:
len(noun_dict)

1695

CountVectorizer 는 document frequency 를 기준으로 대부분 문서에 등장하거나 거의 등장하지 않는 극단적인 단어들을 제거할 수 있습니다. min_df 와 max_df 를 이용하면 되며, df 는 비율입니다. [0, 1] 사이의 값을 입력해야 합니다.

CountVectorizer 의 argument tokenizer 는 str 형식의 문장이 들어왔을 때 이를 list of str 형식의 단어열로 변형하는 함수입니다. 기본값은 lambda x:x.split()으로 띄어쓰기 기준으로 단어를 나눕니다. 우리가 만든 custom_tokenizer 를 이용하여 빈도수가 2 이상인 명사들만을 return 하는 토크나이저로 이를 대체합니다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    tokenizer=custom_tokenizer,
    min_df=0.005,
    max_df=0.95
)

x_sparse = vectorizer.fit_transform(texts)

'''
x_sparse 는 scipy 의 sparse matrix 입니다. Sparse matrix 는 0 값이 많은 행렬에 대하여 0 이 아닌 (i, j) 의 값만을 저장한 
matrix 를 의미합니다. Term frequency matrix 는 very sparse matrix 에 속하므로 dense matrix 보다 sparse matrix 가 훨씬 효율적입니다.
dense matrix 는 문서의 양이 조금만 커도 매우 큰 메모리가 필요합니다.
'''

'\nx_sparse 는 scipy 의 sparse matrix 입니다. Sparse matrix 는 0 값이 많은 행렬에 대하여 0 이 아닌 (i, j) 의 값만을 저장한 \nmatrix 를 의미합니다. Term frequency matrix 는 very sparse matrix 에 속하므로 dense matrix 보다 sparse matrix 가 훨씬 효율적입니다.\ndense matrix 는 문서의 양이 조금만 커도 매우 큰 메모리가 필요합니다.\n'

In [10]:
type(x_sparse)

scipy.sparse.csr.csr_matrix

In [11]:
#vectorizer.vocabulary_ 는 dict 형식으로, {word:index} 입니다.
vectorizer.vocabulary_

{'크리스토퍼': 162,
 '인셉션': 125,
 '감독': 3,
 '인터스텔라': 126,
 '완전': 108,
 '기대': 26,
 '중': 143,
 '작품': 132,
 '감탄': 6,
 '오늘': 106,
 '배우': 63,
 '안': 91,
 '수': 82,
 '영화': 100,
 '극장': 24,
 '거': 11,
 '게': 14,
 '점': 137,
 '개': 8,
 '놀라': 35,
 '이상': 117,
 '자': 129,
 '다': 39,
 '그래비티': 23,
 '생각': 75,
 '데': 43,
 '평점': 165,
 '때': 49,
 '최고': 158,
 '작': 131,
 '동안': 45,
 '소름': 79,
 '번': 64,
 '중간': 144,
 '부분': 66,
 '눈물': 37,
 '장면': 133,
 '분': 68,
 '시간': 86,
 '정도': 138,
 '몰입': 59,
 '끝': 30,
 '설명': 76,
 '예': 104,
 '애': 93,
 '개봉': 9,
 '평': 164,
 '다크': 40,
 '나이트': 31,
 '것': 13,
 '내용': 33,
 '우주': 109,
 '꿀': 29,
 '만큼': 56,
 '실망': 89,
 '듯': 46,
 '일': 127,
 '영화로': 102,
 '천재': 155,
 '느낌': 38,
 '년': 34,
 '전': 135,
 '돈': 44,
 '건': 12,
 '처음': 154,
 '한스': 169,
 '짐머': 151,
 '이건': 114,
 '추천': 160,
 '지구': 146,
 '이': 113,
 '이다': 115,
 '웜홀': 110,
 '각본': 2,
 '조나단': 140,
 '말': 57,
 '필요': 167,
 '사람': 71,
 '예고편': 105,
 '기분': 27,
 '아이맥스': 90,
 '은': 111,
 '상상': 73,
 '대작': 42,
 '영화관': 101,
 '만': 55,
 '볼': 65,
 '한': 168,
 '관심': 21,
 '완

In [12]:
#이를 이용하여 vocab to index, index to vocab 을 만들어 봅니다. 

vocab2int = vectorizer.vocabulary_
vocab2int['영화']

100

In [13]:
# index 는 0 부터 시작합니다. 그러므로 int2vocab 은 dictionary 형태보다 list 형태로 가지고 있어도 좋습니다. 
#이를 위해 sorted 함수를 이용합니다. 100 번째 단어가 '영화'임을 다시 확인할 수 있습니다.

int2vocab = [
    word for word,index in sorted(
        vocab2int.items(), key=lambda x:x[1])
]
int2vocab[100]

'영화'

In [52]:
int2vocab

['5차원',
 '가족',
 '각본',
 '감독',
 '감동',
 '감사',
 '감탄',
 '강',
 '개',
 '개봉',
 '개인',
 '거',
 '건',
 '것',
 '게',
 '경이',
 '고의',
 '공간',
 '공부',
 '과학',
 '관람객',
 '관심',
 '굿',
 '그래비티',
 '극장',
 '급',
 '기대',
 '기분',
 '기억',
 '꿀',
 '끝',
 '나이트',
 '내',
 '내용',
 '년',
 '놀라',
 '눈',
 '눈물',
 '느낌',
 '다',
 '다크',
 '대박',
 '대작',
 '데',
 '돈',
 '동안',
 '듯',
 '등',
 '딸',
 '때',
 '때문',
 '라고',
 '러닝',
 '마음',
 '마지막',
 '만',
 '만큼',
 '말',
 '명작',
 '몰입',
 '물리학',
 '미래',
 '반전',
 '배우',
 '번',
 '볼',
 '부분',
 '부족',
 '분',
 '블랙홀',
 '뿐',
 '사람',
 '사랑',
 '상상',
 '상상력',
 '생각',
 '설명',
 '세계',
 '세상',
 '소름',
 '소재',
 '속',
 '수',
 '수준',
 '스케일',
 '스토리',
 '시간',
 '시공간',
 '신비',
 '실망',
 '아이맥스',
 '안',
 '안보',
 '애',
 '여운',
 '역대',
 '연기',
 '연출',
 '영상',
 '영상미',
 '영화',
 '영화관',
 '영화로',
 '영화인',
 '예',
 '예고편',
 '오늘',
 '완벽',
 '완전',
 '우주',
 '웜홀',
 '은',
 '음악',
 '이',
 '이건',
 '이다',
 '이론',
 '이상',
 '이야기',
 '이영화',
 '이유',
 '이해',
 '인간',
 '인류',
 '인생',
 '인셉션',
 '인터스텔라',
 '일',
 '입',
 '자',
 '자체',
 '작',
 '작품',
 '장면',
 '재미',
 '전',
 '전율',
 '점',
 '정도',
 '정말',
 '조나단',
 '존재',
 '줄',
 '중',
 '중간',


scipy.io 는 sparse matrix 를 읽고 쓰는 기능을 제공합니다. mmwrite 는 sparse matrix 를 filepath 에 저장합니다. 그런데 filepath 의 경로에 directory 가 존재하지 않는다면 오류가 발생합니다. 이를 방지하기 위해서 path 가 존재하는지 확인하고, directory 가 존재하지 않을 경우 이를 만들어야 합니다.

In [14]:
import os
from scipy.io import mmwrite, mmread

if not os.path.exists('tmp/'):
    os.makedirs('tmp')

mmwrite('./tmp/x.mm', x_sparse)

In [15]:
#mmread 는 filepath 의 sparse matrix 를 읽습니다. 그런데 mmread 의 형식은 COO matrix 입니다. 이를 CSR matrix 로 바꾸기 위해 tocsr 함수를 이용합니다.

loaded_x_sparse = mmread('./tmp/x.mm').tocsr()
loaded_x_sparse


<5000x174 sparse matrix of type '<class 'numpy.int64'>'
	with 16161 stored elements in Compressed Sparse Row format>

In [16]:
#vectorizer 는 pickling 으로 저장하여도 좋습니다. vocabulary list 는 따로 list 파일로 저장해도 좋습니다.

In [17]:
import pickle

with open('./tmp/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('./tmp/vocabulary.txt', 'w', encoding='utf-8') as f:
    for vocab in int2vocab:
        f.write('%s\n' % vocab)

In [18]:

with open('./tmp/vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

print(loaded_vectorizer.vocabulary_['영화'])

100


In [19]:
with open('./tmp/vocabulary.txt', encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

5차원
가족
각본
감독
감동


# 뉴스요약하기

In [57]:
from textrank import KeywordSummarizer
from textrank import KeysentenceSummarizer

In [55]:
sents = [
    '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다',
    '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다',
    '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다',
    '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다',
    '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다',
    '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다',
    '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다',
    '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기',
    '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다',
    '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다',
    '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다',
    '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다',
    '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다',
    '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다',
    '총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다',
    '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다',
    '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다',
    '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다',
    '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다',
    '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'
]

In [58]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)

keysents = summarizer.summarize(sents, topk=3)

In [59]:
keysents

[(0,
  1.7010195551998453,
  '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다'),
 (7,
  1.4886043194936454,
  '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기'),
 (8,
  1.4278590385659577,
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다')]

In [60]:
from textrank import KeywordSummarizer

summarizer = KeywordSummarizer(tokenize=komoran_tokenizer, min_count=2, min_cooccurrence=1)
summarizer.summarize(sents, topk=20)

[('용의자/NNP', 3.040833543583403),
 ('사제총/NNP', 2.505798518168069),
 ('성씨/NNP', 2.4254730689696298),
 ('서울/NNP', 2.399522533743009),
 ('경찰/NNG', 2.2541631612221043),
 ('오후/NNG', 2.154778397410354),
 ('폭행/NNG', 1.9019818685234693),
 ('씨/NNB', 1.7517679455874249),
 ('발사/NNG', 1.658959293729613),
 ('맞/VV', 1.618499063577056),
 ('분/NNB', 1.6164369966921637),
 ('번동/NNP', 1.4681655196749035),
 ('현장/NNG', 1.4530182347939307),
 ('시/NNB', 1.408892735491178),
 ('경찰관/NNP', 1.4012941012332316),
 ('조사/NNG', 1.4012941012332316),
 ('일/NNB', 1.3922748983755766),
 ('강북구/NNP', 1.332317291003927),
 ('연합뉴스/NNP', 1.3259099432277819),
 ('이씨/NNP', 1.2869280494707418)]